# Módulo 3 - Modelado por medio de NCF

## Objetivos

## Librerías

In [1]:
import numpy as np
import pandas as pd
from libreco.data import split_by_ratio, DatasetPure, DatasetFeat
from libreco.algorithms import YouTubeRanking
from libreco.evaluation import evaluate

2025-07-05 17:07:13.836011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751753233.882409   75186 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751753233.893301   75186 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751753233.924309   75186 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751753233.924340   75186 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751753233.924342   75186 computation_placer.cc:177] computation placer alr

Instructions for updating:
non-resource variables are not supported in the long term


## Cargado de datos

In [2]:
df = pd.read_csv("m3_merged_df.csv")
df

,Unnamed: 0,ReviewID,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,...,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,0,1,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,...,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,1,2,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,...,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,2,4,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,...,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,3,5,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,...,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,4,6,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,...,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,988,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,...,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,989,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,...,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,990,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,...,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,991,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,...,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


In [3]:
df = df.drop(df.columns[[0]], axis=1)
df

,ReviewID,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,BestTimeToVisit,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,1,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,Oct-Mar,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,2,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,Nov-Feb,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,4,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,Oct-Mar,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,5,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,Sep-Mar,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,6,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,Oct-Mar,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


In [4]:
df.columns

Index(['ReviewID', 'DestinationID_x', 'UserID', 'Rating', 'ReviewText',
       'Name_x', 'State', 'Type', 'Popularity', 'BestTimeToVisit', 'HistoryID',
       'DestinationID_y', 'VisitDate', 'ExperienceRating', 'Name_y', 'Email',
       'Preferences', 'Gender', 'NumberOfAdults', 'NumberOfChildren'],
      dtype='object')

In [5]:
df = df.rename(columns={
    "UserID": "user", 
    "DestinationID_x": "item", 
    "Rating": "label"
})
df.columns

Index(['ReviewID', 'item', 'user', 'label', 'ReviewText', 'Name_x', 'State',
       'Type', 'Popularity', 'BestTimeToVisit', 'HistoryID', 'DestinationID_y',
       'VisitDate', 'ExperienceRating', 'Name_y', 'Email', 'Preferences',
       'Gender', 'NumberOfAdults', 'NumberOfChildren'],
      dtype='object')

In [6]:
user_col = df.pop("user")
df.insert(0, "user", user_col)

item_col = df.pop("item")
df.insert(1, "item", item_col)

label_col = df.pop("label")
df.insert(2, "label", label_col)

In [7]:
sparse_col = [
    "Gender",
    "State",
    "Type",
    "Preferences",
    "BestTimeToVisit"
]
dense_col = [
    "Popularity",
    "NumberOfAdults",
    "NumberOfChildren",
    "ExperienceRating",
]
user_col = ["Gender", "Preferences", "ExperienceRating", "NumberOfAdults", "NumberOfChildren"]
item_col = ["State", "Type", "BestTimeToVisit", "Popularity"]

## Train-Val-Test Split (70%, 20%, 10%)

In [8]:
train_data, eval_data, test_data = split_by_ratio(df, multi_ratios=[0.7, 0.2, 0.1])

## Conversión de data a formato de LibRecommender

In [9]:
train_data, data_info = DatasetFeat.build_trainset(
        train_data, user_col, item_col, sparse_col, dense_col
    )
test_data = DatasetFeat.build_testset(test_data)
print(data_info)  # n_users: 5953, n_items: 3209, data density: 0.4213 %


n_users: 408, n_items: 469, data density: 0.4578 %


## Configuración de NCF

In [10]:
# Hyperparameters
ytb_ranking = YouTubeRanking(
        task="ranking",
        data_info=data_info,
        embed_size=16,
        n_epochs=3,
        lr=1e-4,
        batch_size=512,
        use_bn=True,
        hidden_units=(128, 64, 32),
)


I0000 00:00:1751753245.455981   75186 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2765 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [11]:
"""
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=10,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)
"""

'\nncf = NCF(\n    task="rating",\n    data_info=data_info,\n    loss_type="cross_entropy",\n    embed_size=16,\n    n_epochs=10,\n    lr=1e-3,\n    batch_size=2048,\n    num_neg=1,\n)\n'

## Entrenamiento y evaluación de modelo NCF

In [12]:
ytb_ranking.fit(
        train_data,
        neg_sampling=True,  # sample negative items train and eval data
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"],
)


Training start time: 2025-07-05 17:07:25


TypeError: Exception encountered when calling Flatten.call().

[1mFailed to convert elements of (Dimension(None), Dimension(80)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.[0m

Arguments received by Flatten.call():
  • inputs=tf.Tensor(shape=(?, 5, 16), dtype=float32)

## Pruebas para modelo NCF

In [ ]:
    # predict preference of user 2211 to item 110
print("prediction: ", ytb_ranking.predict(user=221, item=110))
    # recommend 7 items for user 2211
print("recommendation: ", ytb_ranking.recommend_user(user=221, n_rec=7))

    # cold-start prediction
print(
        "cold prediction: ",
        ytb_ranking.predict(user="ccc", item="not item", cold_start="average"),
)
    # cold-start recommendation
print(
        "cold recommendation: ",
        ytb_ranking.recommend_user(user="are we good?", n_rec=7, cold_start="popular"),
)


## Conclusiones